# <font color='blue'>
# Exercise 2.2

<font color='blue'>Cell 1
Importing libraries

In [1]:
install.packages("rpart")
library("rpart") # This R package yields a decision tree with high accuracy

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



<font color='blue'>Cell 2
Reading the data

In [2]:
#Input the data
data <- read.csv(file="cancerdata1.csv", header=TRUE)
data[c(1:3,52:53,102:103),] #Print a few rows of data for each class/label

,ESR1_mg,PGR_mg,BCL2_mg,NAT1_mg,Results
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,5.1,3.5,1.4,0.2,Cured
2,4.9,3.0,1.4,0.2,Cured
3,4.7,3.2,1.3,0.2,Cured
52,6.4,3.2,4.5,1.5,Recurrence
53,6.9,3.1,4.9,1.5,Recurrence
102,5.8,2.7,5.1,1.9,Dead
103,7.1,3.0,5.9,2.1,Dead


<font color='blue'>Cell 3
Dataset verification

In [3]:
#Check the dataset to make sure no data is missing
verify_dataset <- function(dataset) {
  data_found <- 1 #Flag to signal no missing data points were found in a column
  for (each_column in colnames(dataset)) { #Test all columns
    if (any(is.na(dataset[,each_column]))=="TRUE") { #Check for any missing data in column
      cat(paste0("Data missing in Column ", each_column)) #line prints which column is missing data
      data_found <- 0 #Alternative tag
      break() #Terminate loop and ultimately function since the flag data_found is 0
    }
  }

  if (data_found==1) { #Same as flag means no data missing
    print("Dataset is complete. No missing value")
  }

} #end of function

verify_dataset(data) #Test the dataset with the above function

[1] "Dataset is complete. No missing value"


<font color='blue'>Cell 4
Creating testing and training data sets

In [4]:
#Split data into a train and a test set
split_dataset_test_train <- function(data) {
  set.seed(37) #This ensures that we obtain the same selection and is important for reproducibility while building code
  index <- sample(1:nrow(data), 0.7*nrow(data)) #Randomly select 70% of rows
  training_data <- data[index,]
  testing_data <- data[-index,]
  datasets <- list(training_data, testing_data) #Create list called testtrain to hold data sets
  return(datasets) #Return the list
}

testtrain <- split_dataset_test_train(data)
print(testtrain)

[[1]]
    ESR1_mg PGR_mg BCL2_mg NAT1_mg    Results
47      5.1    3.8     1.6     0.2      Cured
24      5.1    3.3     1.7     0.5      Cured
50      5.0    3.3     1.4     0.2      Cured
3       4.7    3.2     1.3     0.2      Cured
109     6.7    2.5     5.8     1.8       Dead
36      5.0    3.2     1.2     0.2      Cured
41      5.0    3.5     1.3     0.3      Cured
127     6.2    2.8     4.8     1.8       Dead
17      5.4    3.9     1.3     0.4      Cured
83      5.8    2.7     3.9     1.2 Recurrence
138     6.4    3.1     5.5     1.8       Dead
67      5.6    3.0     4.5     1.5 Recurrence
132     7.9    3.8     6.4     2.0       Dead
72      6.1    2.8     4.0     1.3 Recurrence
28      5.2    3.5     1.5     0.2      Cured
53      6.9    3.1     4.9     1.5 Recurrence
55      6.5    2.8     4.6     1.5 Recurrence
81      5.5    2.4     3.8     1.1 Recurrence
20      5.1    3.8     1.5     0.3      Cured
105     6.5    3.0     5.8     2.2       Dead
82      5.5    2.4     3.7  

<font color='blue'>Cell 5
Create the classification decision tree and the prediction function using the training data and the gini index

In [5]:
#Create DT and predict outcomes
tree_predict <- function(dataset, train_data) {
  tree <- rpart(formula = Results ~ ., data=train_data, method="class", parms=list(split="gini")) # Creating the desicion tree model using "variety" as classification label, and splitting based on gini index
  tree_pred <- as.data.frame(predict(tree, dataset, type="class"))
  return(tree_pred)
}

training_data <- testtrain[[1]]
tree_pred <- tree_predict(data, training_data)

<font color='blue'>Cell 6
Define and apply the confusion matrix to the training data

In [6]:
#Normalize the confusion maxtrix
normalizeCM <- function(cm){
    cm_norm <- cm
    for (i in 1:3){
        for (j in 1:3){
            cm_norm[i,j] <- cm[i,j]/sum(cm[i,])
        }
    }
    return(round(cm_norm, 2))
}

#Call upon the normalizeCM function and print out the normalized confusion matrix
normalizeCM(table(tree_pred[,1], data$Results))

            
             Cured Dead Recurrence
  Cured       1.00 0.00       0.00
  Dead        0.00 0.98       0.02
  Recurrence  0.00 0.11       0.89

<font color='blue'> Exercise 2.1


<font color='blue'>Cell 7
Function for producing error message for an incorrectly entered test value

In [7]:
## Testing the DT with user inputted values
# The below function inputValue asks the user to enter a numeric value for a certain gene expression level
inputValue <- function(gene_name) {
  # Creates message to prompt user to input the respective value
  message <- paste("Enter ", gene_name, ": ")
  while(TRUE){
    # If below zero or above 10, prints out error message, and prompts the user once more
    # to input a value for the respective feature
    value = as.double(readline(prompt = message))
    if(value < 0 || value > 10){
      warning("Invalid entry, please try again")
    } else {
      # returns the measurement as a double
      return(value)
    }
  }
}

<font color='blue'>Cell 8
Make predictions based on user inputted values

In [8]:
# Create an patient outcome level vector
patient_outcomes <- levels(factor(data$Results))

# Function to predict species
predictPatient <- function(measurements, train_data) {
        #Create a data frame with given input values
        predict_features <- data.frame('ESR1_mg'= measurements[1], 'PGR_mg'= measurements[2],
                                       'BCL2_mg'= measurements[3], 'NAT1_mg'= measurements[4])
        # probabilities for the 3 possible outcomes
        prediction <- tree_predict(predict_features, train_data)
        # Change prediction from numeric to character using species_levels
        patient_pred <- patient_outcomes[as.numeric(prediction)]
        # Inform user of calculated prediction
        cat(paste0("That patient is most likely: ", patient_pred))
}

<font color='blue'>Cell 9
Enter user inputs and display the predicted species

In [9]:
# Call upon the inputValue function four times to obtain the four input values
print('Gene expression levels need to be numeric values between 0 and 10')
ESR1 <-  inputValue('ESR1')
PGR <-  inputValue('PGR')
BCL2 <- inputValue('BCL2')
NAT1 <- inputValue('NAT1')
# create a list with four given measurements
genes = c(ESR1, PGR, BCL2, NAT1)

# Call upon the predictPatient function
predictPatient(genes, training_data)

[1] "Gene expression levels need to be numeric values between 0 and 10"
Enter  ESR1 : 1
Enter  PGR : 1
Enter  BCL2 : 1
Enter  NAT1 : 1
[1] "ESR1 = "
[1] 1
[1] "genes = "
[1] 1 1 1 1
That patient is most likely: Cured